In [1]:
import os
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)
from pyciemss.visuals import plots

In [2]:
DEMO_PATH = "../../notebook/integration_demo/"
ASKENET_PATH_1 = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"
ASKENET_PATH_2 = "../../test/models/AMR_examples/SIDARTHE.amr.json"

ASKENET_PATHS = [ASKENET_PATH_1, ASKENET_PATH_2]

In [3]:
def solution_mapping1(model1_solution: dict) -> dict:
    return model1_solution


def solution_mapping2(model2_solution: dict) -> dict:
    mapped_solution = {}
    mapped_solution["S"] = (
        model2_solution["Susceptible"]
        + model2_solution["Recognized"]
        + model2_solution["Threatened"]
    )*1000

    mapped_solution["I"] = (
        model2_solution["Infected"]
        + model2_solution["Ailing"]
        + model2_solution["Diagnosed"]
    )*1000

    # Model 1 doesn't include dead people, and implicitly assumes that everyone who is infected will recover.
    mapped_solution["R"] = (
        model2_solution["Healed"] + model2_solution["Extinct"]
    )*1000

    return mapped_solution


solution_mappings = [solution_mapping1, solution_mapping2]

## load_and_sample_ensemble_model

In [4]:
weights = [0.5, 0.5]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0, 40, 50]

# Run sampling
ensemble_samples = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints, 
    time_unit="days"
)

# Save results
ensemble_samples.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/sample_results.csv"), index=False)

In [5]:
result = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints,
    time_unit="days", visual_options={"subset":".*_sol"}
)

plots.ipy_display(result["visual"])

## load_and_calibrate_and_sample_ensemble_model

In [6]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0, 50]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    ASKENET_PATHS,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=1000,
    num_iterations=350,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# Save results
result["data"].to_csv(
    os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_sample_results.csv"), index=False
)

plots.ipy_display(result["visual"])

iteration 0: loss = 87.89410543441772
iteration 25: loss = 72.3129997253418
iteration 50: loss = 57.92746663093567
iteration 75: loss = 52.47552466392517
iteration 100: loss = 48.276975870132446
iteration 125: loss = 47.90932023525238
iteration 150: loss = 45.43487513065338
iteration 175: loss = 41.490490794181824
iteration 200: loss = 40.95374882221222
iteration 225: loss = 41.880730628967285
iteration 250: loss = 40.015709400177
iteration 275: loss = 40.85333335399628
iteration 300: loss = 39.826529800891876
iteration 325: loss = 41.415348529815674


In [8]:
plots.ipy_display(result["visual"])